In [ ]:
import numpy as np
import pandas as pd
import cv2
import plotly.io as pio
import time
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
path = '../DMR3/'   # Set path to the directory contaning the data
# Data should have been already cleaned. Data should be organized as follows:
    # Images should be stored sorted by diagnosis and view. For instance, the directory images/healthy/front/ contains frontal images of healthy subjects
    # Clinical data should should be in a csv file, where each, with columns being the different features and rows the different subjects. 
    #     There should be separate files for the healthy and sick groups, and they should be places within a directory called clinical_data/: 
    #     clinical_data/clinical_data_h.csv for healthy patients and clinical_data/clinical_data_s.csv for sick patients


os.mkdir(path + 'Models')
os.mkdir(path + 'Learning_curves')
os.mkdir(path + 'Predictions')
os.mkdir(path + 'Param_tuning')

# Load data

## Thermal images

In [ ]:
## Frontal images
front_images_h = np.asarray([cv2.imread(img,0) for img in os.listdir(path+'images/healthy/front')])
front_images_s = np.asarray([cv2.imread(img,0) for img in os.listdir(path+'images/sick/front')])
front_labels = [0]*len(front_images_h) + [1]*len(front_images_s)

## Left lateral (L90) images
L90_images_h = np.asarray([cv2.imread(img,0) for img in os.listdir(path+'images/healthy/L90')])
L90_images_s = np.asarray([cv2.imread(img,0) for img in os.listdir(path+'images/sick/L90')])
L90_labels = [0]*len(L90_images_h) + [1]*len(L90_images_s)

## Right lateral (R90) images
R90_images_h = np.asarray([cv2.imread(img,0) for img in os.listdir(path+'images/healthy/R90')])
R90_images_s = np.asarray([cv2.imread(img,0) for img in os.listdir(path+'images/sick/R90')])
R90_labels = [0]*len(R90_images_h) + [1]*len(R90_images_s)

In [ ]:
# Shape of thermograms
_,w,h = front_images_h.shape

## Labels (diagnosis)

In [ ]:
labels_h = [0]*len(front_images_h)
labels_s = [1]*len(front_images_s)

## Clinical data

In [ ]:
clinical_data_h = pd.read_csv(path+'clinical_data/clinical_data_h.csv')
clinical_data_s = pd.read_csv(path+'clinical_data/clinical_data_s.csv')
cd_colnames = clinical_data_h.columns

# Pre-processing

## Thermal images

In [ ]:
## Min-max normalization
M = np.concatenate((front_images_h,front_images_s,L90_images_h,L90_images_s,R90_images_h,R90_images_s)).max()
m = np.concatenate((front_images_h,front_images_s,L90_images_h,L90_images_s,R90_images_h,R90_images_s)).min()

front_images_h = ((front_images_h - m) / (M - m)).astype('float32')
front_images_s = ((front_images_s - m) / (M - m)).astype('float32')

L90_images_h = ((L90_images_h - m) / (M - m)).astype('float32')
L90_images_s = ((L90_images_s - m) / (M - m)).astype('float32')

R90_images_h = ((R90_images_h - m) / (M - m)).astype('float32')
R90_images_s = ((R90_images_s - m) / (M - m)).astype('float32')

## Clinical data

In [ ]:
## Normalize
M = pd.concat([clinical_data_h,clinical_data_s]).max().values
M[M<1] = 1
m = pd.concat([clinical_data_h,clinical_data_s]).min().values

clinical_data_h = (clinical_data_h-m)/(M-m)
clinical_data_s = (clinical_data_s-m)/(M-m)

In [ ]:
## Select columns
id_columns_to_delete = [1, 3, 4, 6, 8, 9, 10, 12, 13, 14, 15, 16, 18, 22, 23, 25]
columns_to_delete = cd_colnames[id_columns_to_delete]
clinical_data_h.drop(columns_to_delete,axis=1,inplace=True)
clinical_data_s.drop(columns_to_delete,axis=1,inplace=True)
cd_colnames = list(cd_colnames)
for f in columns_to_delete:
    cd_colnames.remove(f)

In [ ]:
## Convert to numpy array
clinical_data_h = np.asarray(clinical_data_h, dtype=np.float32)
clinical_data_s = np.asarray(clinical_data_s, dtype=np.float32)

# Data split

In [ ]:
def split_data(front_images_h,front_images_s,L90_images_h,L90_images_s,R90_images_h,R90_images_s,clinical_data_h,clinical_data_s,split_rate=0.15):   
    
    ## Randomly shuffle the data
    N_h = len(front_images_h)
    indices_h = np.random.permutation(N_h)
    front_images_h_shuffled = front_images_h[indices_h]
    L90_images_h_shuffled = L90_images_h[indices_h]
    R90_images_h_shuffled = R90_images_h[indices_h]
    clinical_data_h_shuffled = clinical_data_h[indices_h]

    N_s = len(front_images_s)
    indices_s = np.random.permutation(N_s)
    front_images_s_shuffled = front_images_s[indices_s]
    L90_images_s_shuffled = L90_images_s[indices_s]
    R90_images_s_shuffled = R90_images_s[indices_s]
    clinical_data_s_shuffled = clinical_data_s[indices_s]

    ## split healthy and sick sets in train and test
    split_index_h = np.round(N_h * split_rate).astype(int)
    split_index_s = np.round(N_s * split_rate).astype(int)

    train_front_h = front_images_h_shuffled[split_index_h : ]
    test_front_h = front_images_h_shuffled[ : split_index_h]
    train_L90_h = L90_images_h_shuffled[split_index_h : ]
    test_L90_h = L90_images_h_shuffled[ : split_index_h]
    train_R90_h = R90_images_h_shuffled[split_index_h : ]
    test_R90_h = R90_images_h_shuffled[ : split_index_h]
    train_cd_h = clinical_data_h_shuffled[split_index_h : ]
    test_cd_h = clinical_data_h_shuffled[ : split_index_h]

    train_front_s = front_images_s_shuffled[split_index_s : ]
    test_front_s = front_images_s_shuffled[ : split_index_s]
    train_L90_s = L90_images_s_shuffled[split_index_s : ]
    test_L90_s = L90_images_s_shuffled[ : split_index_s]
    train_R90_s = R90_images_s_shuffled[split_index_s : ]
    test_R90_s = R90_images_s_shuffled[ : split_index_s]
    train_cd_s = clinical_data_s_shuffled[split_index_s : ]
    test_cd_s = clinical_data_s_shuffled[ : split_index_s]
    
    ## create labels
    train_labels_h = [0]*len(train_front_h)
    test_labels_h = [0]*len(test_front_h)

    train_labels_s = [1]*len(train_front_s)
    test_labels_s = [1]*len(test_front_s)
    
    ## create train and test sets
    train_front = np.concatenate((train_front_h, train_front_s))
    train_L90 = np.concatenate((train_L90_h, train_L90_s))
    train_R90 = np.concatenate((train_R90_h, train_R90_s))
    train_cd = np.concatenate((train_cd_h, train_cd_s))
    train_labels = np.concatenate((train_labels_h, train_labels_s))
    
    test_front = np.concatenate((test_front_h, test_front_s))
    test_L90 = np.concatenate((test_L90_h, test_L90_s))
    test_R90 = np.concatenate((test_R90_h, test_R90_s))
    test_cd = np.concatenate((test_cd_h, test_cd_s))
    test_labels = np.concatenate((test_labels_h, test_labels_s))
    
    ## Randomly shuffle data
    indices_train = np.random.permutation(len(train_labels))
    train_front = train_front[indices_train]
    train_L90 = train_L90[indices_train]
    train_R90 = train_R90[indices_train]
    train_cd = train_cd[indices_train]
    train_labels = train_labels[indices_train]
    
    indices_test = np.random.permutation(len(test_labels))
    test_front = test_front[indices_test]
    test_L90 = test_L90[indices_test]
    test_R90 = test_R90[indices_test]
    test_cd = test_cd[indices_test]
    test_labels = test_labels[indices_test]
    
    train_data = [train_front,train_L90,train_R90,train_cd,train_labels]
    test_data = [test_front,test_L90,test_R90,test_cd,test_labels]
    
    return train_data,test_data

In [ ]:
train_data,test_data = split_data(front_images_h,front_images_s,L90_images_h,L90_images_s,R90_images_h,R90_images_s,clinical_data_h,clinical_data_s,split_rate=0.15)
train_front,train_L90,train_R90,train_cd,train_labels = train_data
test_front,test_L90,test_R90,test_cd,test_labels = test_data

In [ ]:
n_sick = train_labels.sum()
n_healthy = len(train_labels) - n_sick
rate_train = n_healthy / n_sick

# Training

In [ ]:
# Number of epochs
EPOCHS = 30 

In [ ]:
## Loss function
def weighted_error(y_test, y_test_pred, rate=20):
            
    # False negatives (y = 1, y_pred = 0)
    fn = np.sum(np.greater(y_test, y_test_pred))

    # False positives (y = 0, y_pred = 1)
    fp = np.sum(np.less(y_test, y_test_pred))
    
    return fn*rate + fp

In [ ]:
def plot_training(train_history,val_history,num_epochs):
    font_title = 12
    font_legend = 10
    
    # Visualize the training results
    epochs_range = range(num_epochs)
    plt.figure(figsize=(30, 10))
    
    train_loss,train_acc,train_AUC = train_history
    val_loss,val_acc,val_AUC = val_history
    
    plt.subplot(1, 3, 1)
    plt.plot(epochs_range, train_loss, label='Train Loss')
    plt.plot(epochs_range, val_loss, label='Test Loss')
    plt.legend(loc='upper right',fontsize=font_legend)
    plt.title('Train and Test Loss',fontsize=font_title)
    
    plt.subplot(1, 3, 2)
    plt.plot(epochs_range, train_acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right',fontsize=font_legend)
    plt.title('Train and Test Accuracy',fontsize=font_title)
    
    plt.subplot(1, 3, 3)
    plt.plot(epochs_range, train_AUC, label='Train AUC')
    plt.plot(epochs_range, val_AUC, label='Test AUC')
    plt.legend(loc='lower right',fontsize=font_legend)
    plt.title('Train and Test ROC AUC',fontsize=font_title)

    #plt.show()
    
    return

In [ ]:
from sklearn.metrics import accuracy_score

def numeric_results(ground_truth,preds_raw):
    
    # Binary Cross Entropy
    bce = tf.keras.metrics.BinaryCrossentropy  ()
    bce.update_state(ground_truth, preds_raw)
    
    # ROC AUC
    auc = tf.keras.metrics.AUC()
    auc.update_state(ground_truth, preds_raw)
    
    # Precision
    precision = tf.keras.metrics.Precision()
    precision.update_state(ground_truth, preds_raw)
    
    # Recall
    recall = tf.keras.metrics.Recall()
    recall.update_state(ground_truth, preds_raw)
    
    TN, FP, FN, TP = confusion_matrix(ground_truth, np.round(preds_raw)).ravel()
    results = {
        'BCELoss':bce.result().numpy(),
        'Accuracy':accuracy_score(ground_truth, np.round(preds_raw)),
        'TP':TP,
        'FP':FP,
        'TN':TN,
        'FN':FN,
        'sensitivity':TP/(TP+FN),
        'specificity':TN/(TN+FP),
        'G-mean':np.sqrt((TP/(TP+FN))*(TN/(TN+FP))),
        'precision':precision.result().numpy(),
        'recall':recall.result().numpy(),
        'F1':2*precision.result().numpy()*recall.result().numpy()/(precision.result().numpy()+recall.result().numpy()),
        'ROC AUC':auc.result().numpy(),
        'WE':weighted_error(ground_truth, np.round(preds_raw))
    }
    return results

In [ ]:
## Bootstraping
from sklearn.metrics import roc_auc_score
def bootstrapping(y_true,y_pred):
    # Bootstrap resampling
    n_bootstraps = 1000  # Number of bootstrap samples
    auc_scores = []      # List to store AUC scores from each bootstrap sample
    rng = np.random.RandomState(42)  # Random state for reproducibility
    
    auc = tf.keras.metrics.AUC()
    
    # Compute the AUC on the original data
    auc.update_state(y_true, y_pred)
    observed_auc = auc.result().numpy()
    
    count = 0
    while count < n_bootstraps:
    #for _ in range(n_bootstraps):
        # Generate bootstrap sample indices
        indices = rng.choice(len(y_true), size=len(y_true), replace=True)

        #if len(np.unique(y_true[indices])==1): print('A')
        if len(np.unique(y_true[indices]))> 1:
        
            # Compute AUC on bootstrap sample
            auc.update_state(y_true[indices], y_pred[indices])
            auc_bootstrap = auc.result().numpy()
            auc_scores.append(auc_bootstrap)
            
            count = count+1
            
    # Calculate statistics from bootstrap samples
    mean_auc = np.mean(auc_scores)
    std_auc = np.std(auc_scores)
    ci_lower, ci_upper = np.percentile(auc_scores, [2.5, 97.5])  # 95% confidence interval
    
    # Calculate p-value (two-tailed)
    p_value = np.mean(np.abs(auc_scores - observed_auc) >= np.abs(observed_auc))

    print(f"Mean AUC: {mean_auc:.4f}")
    print(f"Standard Deviation of AUC: {std_auc:.4f}")
    print(f"95% Confidence Interval of AUC: [{ci_lower:.4f}, {ci_upper:.4f}]")
    print(f"p-value of AUC: {p_value:.4f}")
    
    return mean_auc,std_auc,ci_lower,ci_upper,p_value

In [ ]:
## Get the number of parameters of each model
def num_parameters_classifiers(classifier_type,cd_classifier):
        
    if classifier_type == 'WeightedVoting':
        n_parameters = 1
        
    if classifier_type == 'SVC':
        n_support_vectors = len(cd_classifier.support_vectors_)
        n_coefficients = len(cd_classifier.dual_coef_[0])
        n_parameters = n_support_vectors + n_coefficients
        
    elif classifier_type == 'DT':
        n_parameters = cd_classifier.tree_.node_count
        
    elif classifier_type == 'RF':
        n_trees = len(cd_classifier.estimators_)
        n_parameters = sum(tree.tree_.node_count for tree in cd_classifier.estimators_)
        
    elif classifier_type == 'DT_AdaBoost':
        n_estimators = len(cd_classifier.estimators_)
        n_parameters = sum(estimator.tree_.node_count for estimator in cd_classifier.estimators_)
        
    elif classifier_type == 'RF_AdaBoost':
        n_estimators = len(cd_classifier.estimators_)
        n_parameters = sum(
            sum(tree.tree_.node_count for tree in estimator.estimators_)
            for estimator in cd_classifier.estimators_
        )
        
    elif classifier_type == 'SGD':
            n_parameters = classifier.coef_.size + classifier.intercept_.size
            
    elif classifier_type == 'NN':
        n_parameters = classifier.coef_.size + classifier.intercept_.size

    else:
        print('Error')
        n_parameters = None
        
    return n_parameters

## Parameter tuning

In [ ]:
import optuna

In [ ]:
k = 4  # k-fold cross-validation
n_trials = 30  # Trials for hyperparameter tuning

In [ ]:
def get_colnames(param_names):
    colnames = ['Model','k','target'] + param_names + ['tuningTime','bestValue']
    return colnames

In [ ]:
def create_crossval_subsets(front,L90,R90,cd,labels,k):
    # Create train+val subsets for k-fold cross-validation
    fold_size = len(labels) // k
    
    train_val_subsets = []
    for i in range(k):
        val_front = front[i*fold_size:(i+1)*fold_size]
        val_L90 = L90[i*fold_size:(i+1)*fold_size]
        val_R90 = R90[i*fold_size:(i+1)*fold_size]
        val_cd = cd[i*fold_size:(i+1)*fold_size]
        val_labels = labels[i*fold_size:(i+1)*fold_size]
        
        train_front = np.concatenate((front[0:i*fold_size],front[(i+1)*fold_size::]))
        train_L90 = np.concatenate((L90[0:i*fold_size],L90[(i+1)*fold_size::]))
        train_R90 = np.concatenate((R90[0:i*fold_size],R90[(i+1)*fold_size::]))
        train_cd = np.concatenate((cd[0:i*fold_size],cd[(i+1)*fold_size::]))
        train_labels = np.concatenate((labels[0:i*fold_size],labels[(i+1)*fold_size::]))
        
        train_data = [train_front,train_L90,train_R90,train_cd,train_labels]
        val_data = [val_front,val_L90,val_R90,val_cd,val_labels]
        
        train_val_subsets.append([train_data,val_data])
    
    return train_val_subsets

In [ ]:
import sklearn
from sklearn.model_selection import GridSearchCV

def classifier_tuneHyperparameters(estimator,param_grid,X_train,y_train):
    #cost_scorer = make_scorer(weighted_error, greater_is_better=False)
    cost_scorer = 'roc_auc'

    # Tune hyperparameters with 4-fold cross-validation on training set
    classifier = GridSearchCV(estimator, param_grid, scoring=cost_scorer, cv=4).fit(X_train, y_train)
    parameters = classifier.best_params_
    results_cv = classifier.cv_results_
    
    return classifier, parameters, results_cv

# Clinical data distribution across folds

In [ ]:
values = {feat : np.unique(np.concatenate((clinical_data_h,clinical_data_s))[:,i]) for i,feat in enumerate(cd_colnames) if 'age' not in feat}

In [ ]:
n_healthy_train = sum(train_labels==0)
n_sick_train = sum(train_labels==1)
n_healthy_test = sum(test_labels==0)
n_sick_test = sum(test_labels==1)
    
count_train_h,count_train_s,count_test_h,count_test_s = {},{},{},{}
for i,feat in enumerate(cd_colnames):
    if 'age' not in feat:
        # Categorical feature
        for v in values[feat]:
            count_train_h[feat+': '+str(v)+' (%)'] = sum(train_cd[train_labels==0][:,i] == v) / n_healthy_train
            count_test_h[feat+': '+str(v)+' (%)'] = sum(test_cd[test_labels==0][:,i] == v) / n_healthy_test
            count_train_s[feat+': '+str(v)+' (%)'] = sum(train_cd[train_labels==1][:,i] == v) / n_sick_train
            count_test_s[feat+': '+str(v)+' (%)'] = sum(test_cd[test_labels==1][:,i] == v) / n_sick_test
    else:
        # Numerical feature
        count_train_h[feat+' [mean,std]'] = [train_cd[train_labels==0][:,i].mean(),train_cd[train_labels==0][:,i].std()]
        count_test_h[feat+' [mean,std]'] = [test_cd[test_labels==0][:,i].mean(),test_cd[test_labels==0][:,i].std()]
        count_train_s[feat+' [mean,std]'] = [train_cd[train_labels==1][:,i].mean(),train_cd[train_labels==1][:,i].std()]
        count_test_s[feat+' [mean,std]'] = [test_cd[test_labels==1][:,i].mean(),test_cd[test_labels==1][:,i].std()]
            
distribution = {'train_healthy' : count_train_h,
                'test_healthy' : count_test_h,
                'train_sick' : count_train_s,
                'test_sick' : count_test_s}

pd.set_option('display.width', 1000)
print(pd.DataFrame(distribution))

# Clinical Data (CD) classifier

In [ ]:
import pickle
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, make_scorer, roc_auc_score

In [ ]:
models = ['SVC','DT','RF','DT_AdaBoost','RF_AdaBoost']

## Tune hyperparameters

In [ ]:
cd_params = {}
for classifier_type in models:
    print(classifier_type)
    if classifier_type == 'SVC':
        # Support Vector Machine (SVM) classifier
        estimator = SVC(class_weight={0: 1, 1: rate_train})
        param_grid = [{'C': range(1, 1000, 10), 'kernel': ['linear', 'rbf', 'sigmoid'], 'gamma': ['scale', 'auto', 0.001, 0.0001, 0.00001]}]
        
    elif classifier_type in ['DT','DT_AdaBoost']:
        # Decision Tree (DT) classifier
        estimator = DecisionTreeClassifier(class_weight={0: 1, 1: rate_train})
        param_grid = [{'ccp_alpha' : np.arange(0, 0.1, 0.005),
                       'criterion': ["gini"],
                       'max_depth' : [None, 1, 10, 15],
                       'max_features': ['auto'],
                       'max_leaf_nodes': [None, 3, 6, 9],
                       'min_samples_leaf': [2, 3, 4],
                       'min_samples_split' : [3, 10, 15],
                       'min_weight_fraction_leaf' : np.arange(0.1, 0.5, 0.01),
                       'splitter': ['best']}]

    else:
        # Random Forest (RF) classifier
        estimator = RandomForestClassifier(class_weight={0: 1, 1: rate_train})
        param_grid = {'n_estimators': [int(x) for x in np.linspace(start=200, stop=2000, num=10)],  # Number of trees in random forest
                      'max_features': ['auto', 'sqrt', 'log2'],  # Number of features to consider at every split
                      'max_depth': [int(x) for x in np.linspace(10, 110, num=11)]+['None'],  # Maximum number of levels in tree
                      'min_samples_split': [2, 3, 5, 10],  # Minimum number of samples required to split a node
                      'min_samples_leaf': [1, 2, 3, 4],  # Minimum number of samples required at each leaf node
                      'bootstrap': [True, False]}  # Method of selecting samples for training each tree
    
    classifier, parameters, results_cv = classifier_tuneHyperparameters(estimator, param_grid, train_cd, train_labels)
    cd_params[classifier_type] = parameters
    
    print('Tuned parameters:'), print(parameters), print()
    
    # Evaluate
    print('TRAIN:'), print(numeric_results(train_labels,classifier.predict(train_cd)))
    print('TEST:'), print(numeric_results(test_labels,classifier.predict(test_cd)))
    
    print('#'*150), print()

## Train classifiers

In [ ]:
def train_cd_classifier(train_cd,train_labels,test_cd,test_labels,classifier_type,parameters):  
    
    n_sick = train_labels.sum()
    n_healthy = len(train_labels) - n_sick
    rate_train = n_healthy / n_sick
    
    ## Initialize the estimator
    if classifier_type == 'SVC':
        # Support Vector Machine (SVM) classifier
        estimator = SVC(class_weight={0: 1, 1: rate_train}, probability=True)
        
    elif classifier_type in ['DT','DT_AdaBoost']:
        # Decision Tree (DT) classifier
        #parameters = {'ccp_alpha': 0.0, 'criterion': 'gini', 'max_depth': 10, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_samples_leaf': 3, 
        #              'min_samples_split': 10, 'min_weight_fraction_leaf': 0.13999999999999999, 'splitter': 'best'}
        estimator = DecisionTreeClassifier(class_weight={0: 1, 1: rate_train})
    else:
        # Random Forest (RF) classifier
        #parameters = {'n_estimators': 1200, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 110, 'bootstrap': False}
        estimator = RandomForestClassifier(class_weight={0: 1, 1: rate_train})
            
    estimator.set_params(**parameters)  
        
    ## Train the classifier
    tic = time.time()
    if 'AdaBoost' in classifier_type:
        classifier = AdaBoostClassifier(estimator).fit(train_cd, train_labels)
    else:
        classifier = estimator.fit(train_cd, train_labels)
    train_time = time.time() - tic

    ## Predictions
    tic = time.time()
    predictions_train = classifier.predict_proba(train_cd)[:,1]
    inference_time = time.time()-tic

    tic = time.time()
    predictions_test = classifier.predict_proba(test_cd)[:,1]
    inference_time = inference_time + (time.time()-tic)

    inference_time = inference_time/(len(predictions_train)+len(predictions_test))
        
    return classifier,train_time,inference_time,predictions_train,predictions_test

In [ ]:
## Train classifiers
for classifier_type in models:
    print(classifier_type), print()
        
    ## Train the classifier
    classifier,train_time,inference_time,predictions_train,predictions_test = train_cd_classifier(train_cd,train_labels,
                                                                                                  test_cd,test_labels,
                                                                                                  classifier_type,{})#cd_params[classifier_type])
    
    ## Save the model
    with open(path+'Models/cd_'+classifier_type+'.pkl','wb') as f:
        pickle.dump(classifier,f)
        
    np.save(path+'Predictions/cd_'+classifier_type+'_train.npy',predictions_train)
    np.save(path+'Predictions/cd_'+classifier_type+'_test.npy',predictions_test)
    
    print('TRAIN - ',str(train_time),'s'), print(numeric_results(train_labels,predictions_train))
    print('TEST - ', str(inference_time),'s'), print(numeric_results(test_labels,predictions_test))
    print()
    print('Bootstrapping')
    print(bootstrapping(test_labels,predictions_test))
    print()
    print('Number of parameters: ', num_parameters_classifiers(classifier_type,classifier))
    print()


## Select best classifier

In [ ]:
cd_model = 'DT_AdaBoost'

# Thermal image classifier

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation
from tensorflow.keras.layers import Flatten, concatenate, RepeatVector
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras import Input

In [ ]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.utils import metrics_utils
from tensorflow.python.keras.utils.generic_utils import to_list
from tensorflow.python.ops import init_ops
from tensorflow.python.ops import math_ops

class Specificity(tf.keras.metrics.Metric):
    """Computes the specificity of the predictions with respect to the labels.

    This metric creates two local variables, `true_negatives` and
    `false_positives`, that are used to compute the specificity. This value is
    ultimately returned as `specificity`, an idempotent operation that simply divides
    `true_negatives` by the sum of `true_negatives` and `false_positives`.

    If `sample_weight` is `None`, weights default to 1.
    Use `sample_weight` of 0 to mask values.

    If `top_k` is set, recall will be computed as how often on average a class
    among the labels of a batch entry is in the top-k predictions.

    If `class_id` is specified, we calculate specificity by considering only the
    entries in the batch for which `class_id` is in the label, and computing the
    fraction of them for which `class_id` is above the threshold and/or in the
    top-k predictions.
    """

    def __init__(self,
               thresholds=None,
               top_k=None,
               class_id=None,
               name='specificity',
               dtype=None):
        """Creates a `specificity` instance.

        Args:
          thresholds: (Optional) A float value or a python list/tuple of float
            threshold values in [0, 1]. A threshold is compared with prediction
            values to determine the truth value of predictions (i.e., above the
            threshold is `true`, below is `false`). One metric value is generated
            for each threshold value. If neither thresholds nor top_k are set, the
            default is to calculate recall with `thresholds=0.5`.
          top_k: (Optional) Unset by default. An int value specifying the top-k
            predictions to consider when calculating recall.
          class_id: (Optional) Integer class ID for which we want binary metrics.
            This must be in the half-open interval `[0, num_classes)`, where
            `num_classes` is the last dimension of predictions.
          name: (Optional) string name of the metric instance.
          dtype: (Optional) data type of the metric result.
        """
        super(Specificity, self).__init__(name=name, dtype=dtype)
        self.init_thresholds = thresholds
        self.top_k = top_k
        self.class_id = class_id

        default_threshold = 0.5 if top_k is None else metrics_utils.NEG_INF
        self.thresholds = metrics_utils.parse_init_thresholds(
            thresholds, default_threshold=default_threshold)
        self.true_negatives = self.add_weight(
            'true_negatives',
            shape=(len(self.thresholds),),
            initializer=init_ops.zeros_initializer)
        self.false_positives = self.add_weight(
            'false_positives',
            shape=(len(self.thresholds),),
            initializer=init_ops.zeros_initializer)

    def update_state(self, y_true, y_pred, sample_weight=None):
        """Accumulates true negative and false positive statistics.

        Args:
          y_true: The ground truth values, with the same dimensions as `y_pred`.
            Will be cast to `bool`.
          y_pred: The predicted values. Each element must be in the range `[0, 1]`.
          sample_weight: Optional weighting of each example. Defaults to 1. Can be a
            `Tensor` whose rank is either 0, or the same rank as `y_true`, and must
            be broadcastable to `y_true`.

        Returns:
          Update op.
        """
        return metrics_utils.update_confusion_matrix_variables(
            {
                metrics_utils.ConfusionMatrix.TRUE_NEGATIVES: self.true_negatives,
                metrics_utils.ConfusionMatrix.FALSE_POSITIVES: self.false_positives
            },
            y_true,
            y_pred,
            thresholds=self.thresholds,
            top_k=self.top_k,
            class_id=self.class_id,
            sample_weight=sample_weight)

    def result(self):
        result = math_ops.div_no_nan(self.true_negatives,
                                     self.true_negatives + self.false_positives)
        return result[0] if len(self.thresholds) == 1 else result

    def reset_states(self):
        num_thresholds = len(to_list(self.thresholds))
        K.batch_set_value(
            [(v, np.zeros((num_thresholds,))) for v in self.variables])
        
    def get_config(self):
        config = {
            'thresholds': self.init_thresholds,
            'top_k': self.top_k,
            'class_id': self.class_id
        }
        base_config = super(Specificity, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


class WeightedError(tf.keras.metrics.Metric):
    """Computes the Weighted Error of the predictions with respect to the labels: FP + rate * FN

    This metric creates two local variables, `false_negatives` and
    `false_positives`, that are used to compute the weighted error. This value is
  ultimately returned as `weighted-error`, an idempotent operation that simply multiplies
    `false_negatives` by 'rate' and sums this value with `false_positives`.

    If `sample_weight` is `None`, weights default to 1.
    Use `sample_weight` of 0 to mask values.

    If `top_k` is set, weighted error will be computed as how often on average a class
    among the labels of a batch entry is in the top-k predictions.

    If `class_id` is specified, we calculate recall by considering only the
    entries in the batch for which `class_id` is in the label, and computing the
    fraction of them for which `class_id` is above the threshold and/or in the
    top-k predictions.
    """

    def __init__(self,
               thresholds=None,
               top_k=None,
               class_id=None,
               rate=None,
               name='weighted-error',
               dtype=None):
        """Creates a `weighted-error` instance.

        Args:
          thresholds: (Optional) A float value or a python list/tuple of float
            threshold values in [0, 1]. A threshold is compared with prediction
            values to determine the truth value of predictions (i.e., above the
            threshold is `true`, below is `false`). One metric value is generated
            for each threshold value. If neither thresholds nor top_k are set, the
            default is to calculate recall with `thresholds=0.5`.
          top_k: (Optional) Unset by default. An int value specifying the top-k
            predictions to consider when calculating recall.
          class_id: (Optional) Integer class ID for which we want binary metrics.
            This must be in the half-open interval `[0, num_classes)`, where
            `num_classes` is the last dimension of predictions.
           rate: Integer class ID. The rate of the error.
          name: (Optional) string name of the metric instance.
          dtype: (Optional) data type of the metric result.
        """
        super(WeightedError, self).__init__(name=name, dtype=dtype)
        self.init_thresholds = thresholds
        self.top_k = top_k
        self.class_id = class_id
        self.rate = rate

        default_threshold = 0.5 if top_k is None else metrics_utils.NEG_INF
        self.thresholds = metrics_utils.parse_init_thresholds(
            thresholds, default_threshold=default_threshold)
        self.false_negatives = self.add_weight(
            'false_negatives',
            shape=(len(self.thresholds),),
            initializer=init_ops.zeros_initializer)
        self.false_positives = self.add_weight(
            'false_positives',
            shape=(len(self.thresholds),),
            initializer=init_ops.zeros_initializer)

    def update_state(self, y_true, y_pred, sample_weight=None):
        """Accumulates true positive and false negative statistics.

        Args:
          y_true: The ground truth values, with the same dimensions as `y_pred`.
            Will be cast to `bool`.
          y_pred: The predicted values. Each element must be in the range `[0, 1]`.
          sample_weight: Optional weighting of each example. Defaults to 1. Can be a
            `Tensor` whose rank is either 0, or the same rank as `y_true`, and must
            be broadcastable to `y_true`.

        Returns:
          Update op.
        """
        return metrics_utils.update_confusion_matrix_variables(
            {
                metrics_utils.ConfusionMatrix.FALSE_NEGATIVES: self.false_negatives,
                metrics_utils.ConfusionMatrix.FALSE_POSITIVES: self.false_positives
            },
            y_true,
            y_pred,
            thresholds=self.thresholds,
            top_k=self.top_k,
            class_id=self.class_id,
            sample_weight=sample_weight)

    def result(self):
        result = self.false_positives + math_ops.multiply_no_nan(self.false_negatives, self.rate)
        return result[0] if len(self.thresholds) == 1 else result

    def reset_states(self):
        num_thresholds = len(to_list(self.thresholds))
        K.batch_set_value(
            [(v, np.zeros((num_thresholds,))) for v in self.variables])

    def get_config(self):
        config = {
            'thresholds': self.init_thresholds,
            'top_k': self.top_k,
            'class_id': self.class_id,
            'rate': self.rate
        }
        base_config = super(WeightedError, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
def num_parameters_img(classifier):
    ## Get the number of parameters of each model
    total_params = np.sum([np.prod(v.get_shape().as_list()) for v in classifier.variables])
    trainable_params = np.sum([np.prod(v.get_shape().as_list()) for v in classifier.trainable_variables])
    return total_params,trainable_params

## Proposed multi-input CNN

In [ ]:
## 3-block model, one for each view
def image_classifier_model_3blocks(a, b, c):
    """
    CNN with frontal, L90 and R90 thermos of shape (a, b, c) and clinical data.
    big_cd is True when there are more than 16 features, otherwise is False
    """

    front_input = Input(shape=(a, b, c))
    front = Conv2D(32, (3, 3), activation='relu', input_shape=(a, b, c))(front_input)
    front = Conv2D(64, (5, 5), strides=2, activation='relu')(front)
    front = MaxPooling2D((3, 3))(front)
    front = Conv2D(64, (3, 3), activation='relu')(front)  
    front = Conv2D(64, (5, 5), strides=2, activation='relu')(front)  
    front = Conv2D(64, (3, 3), activation='relu')(front) 
    front = MaxPooling2D((2, 2))(front)
    front = Conv2D(128, (3, 3), activation='relu')(front) 
    front = Conv2D(128, (3, 3), activation='relu')(front) # (5, 5), strides=2
    front = Conv2D(128, (3, 3), activation='relu')(front)  # new
    #front = MaxPooling2D((2, 2))(front)  
    front = Conv2D(128, (5, 5), strides=2, activation='relu')(front) #strides=2
    front = MaxPooling2D((2, 2))(front)  
    front = Flatten()(front) 
    front = Dense(512, use_bias=False)(front)   #256
    front = BatchNormalization()(front)
    front = Activation('relu')(front)
    front = Dropout(0.5)(front) 
    front = Dense(128, use_bias=False)(front) # activation='relu'
    front = BatchNormalization()(front)
    front = Activation('relu')(front)
    front = Dropout(0.5)(front) 
    front = Dense(64, activation='relu')(front) 
    front = Dense(1, activation='sigmoid')(front) 


    L90_input = Input(shape=(a, b, c))
    L90 = Conv2D(32, (3, 3), activation='relu', input_shape=(a, b, c))(L90_input)
    L90 = Conv2D(32, (5, 5), strides=2, activation='relu')(L90) 
    L90 = MaxPooling2D((2, 2))(L90)
    L90 = Conv2D(64, (3, 3), activation='relu')(L90) 
    L90 = Conv2D(64, (3, 3), activation='relu')(L90) 
    L90 = Conv2D(64, (3, 3), activation='relu')(L90) #new
    L90 = MaxPooling2D((2, 2))(L90)
    #L90 = Conv2D(128, (3, 3), activation='relu')(L90) 
    L90 = Conv2D(128, (3, 3), activation='relu')(L90)
    L90 = Conv2D(128, (3, 3), activation='relu')(L90) 
    #L90 = MaxPooling2D((2, 2))(L90)  
    L90 = Conv2D(128, (3, 3), activation='relu')(L90) 
    L90 = MaxPooling2D((2, 2))(L90)
    L90 = Flatten()(L90)
    L90 = Dense(512, use_bias=False)(L90)   #256
    L90 = BatchNormalization()(L90)
    L90 = Activation('relu')(L90)
    L90 = Dropout(0.5)(L90) 
    L90 = Dense(128, use_bias=False)(L90) # activation='relu'
    L90 = BatchNormalization()(L90)
    L90 = Activation('relu')(L90)
    L90 = Dropout(0.5)(L90) 
    L90 = Dense(64, activation='relu')(L90) 
    L90 = Dense(1, activation='sigmoid')(L90) 

    R90_input = Input(shape=(a, b, c))
    R90 = Conv2D(32, (3, 3), activation='relu', input_shape=(a, b, c))(R90_input)
    R90 = Conv2D(32, (5, 5), strides=2, activation='relu')(R90)   
    R90 = MaxPooling2D((2, 2))(R90) 
    R90 = Conv2D(64, (3, 3), activation='relu')(R90)  
    R90 = Conv2D(64, (3, 3), activation='relu')(R90) 
    R90 = Conv2D(64, (3, 3), activation='relu')(R90)  #new
    R90 = MaxPooling2D((2, 2))(R90)
    R90 = Conv2D(128, (3, 3), activation='relu')(R90) 
    R90 = Conv2D(128, (3, 3), activation='relu')(R90)
    #R90 = Conv2D(128, (3, 3), activation='relu')(R90)  
    #R90 = MaxPooling2D((2, 2))(R90)  
    R90 = Conv2D(128, (3, 3), activation='relu')(R90)  
    R90 = MaxPooling2D((2, 2))(R90) 
    R90 = Flatten()(R90)
    R90 = Dense(512, use_bias=False)(R90)   #256
    R90 = BatchNormalization()(R90)
    R90 = Activation('relu')(R90)
    R90 = Dropout(0.5)(R90) 
    R90 = Dense(128, use_bias=False)(R90) # activation='relu'
    R90 = BatchNormalization()(R90)
    R90 = Activation('relu')(R90)
    R90 = Dropout(0.5)(R90) 
    R90 = Dense(64, activation='relu')(R90) 
    R90 = Dense(1, activation='sigmoid')(R90) 

    concatenated = concatenate([front, L90, R90], axis=-1)

    X = Dense(1, activation='sigmoid')(concatenated)   

    ## create the model
    model = Model([front_input, L90_input, R90_input], X)  

    return model 

In [ ]:
## 1-block model, with each view as an input channel
def image_classifier_model_1block(a, b):
    """
    CNN with frontal, L90 and R90 thermos of shape (a, b, c) and clinical data.
    big_cd is True when there are more than 16 features, otherwise is False
    """

    # Inputs
    front_input = Input(shape=(a, b, 1))
    L90_input = Input(shape=(a, b, 1))
    R90_input = Input(shape=(a, b, 1))
    stacked_input = tf.concat([front_input, L90_input, R90_input], axis=-1)
    
    #img_input = Input(shape=(a, b, c))
    x = Conv2D(32, (3, 3), activation='relu', input_shape=(a, b, 3))(stacked_input)
    x = Conv2D(64, (5, 5), strides=2, activation='relu')(x)
    x = MaxPooling2D((3, 3))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)  
    x = Conv2D(64, (5, 5), strides=2, activation='relu')(x)  
    x = Conv2D(64, (3, 3), activation='relu')(x) 
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x) 
    x = Conv2D(128, (3, 3), activation='relu')(x) # (5, 5), strides=2
    x = Conv2D(128, (3, 3), activation='relu')(x)  # new
    #x = MaxPooling2D((2, 2))(x)  
    x = Conv2D(128, (5, 5), strides=2, activation='relu')(x) #strides=2
    x = MaxPooling2D((2, 2))(x)  
    x = Flatten()(x) 
    x = Dense(512, use_bias=False)(x)   #256
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x) 
    x = Dense(128, use_bias=False)(x) # activation='relu'
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x) 
    x = Dense(64, activation='relu')(x) 
    x = Dense(1, activation='sigmoid')(x) 

    X = Dense(1, activation='sigmoid')(x)   

    ## create the model
    model = Model([front_input, L90_input, R90_input], X)  
    
    return model 

### Tune hyperparameters

In [ ]:
def objective(trial, model, front, L90, R90, cd, labels, k, EPOCHS=30):  
    
    ## Initialize model
    init_weights = model.get_weights()
    
    ## Hyperparameters
    # Learning rate
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    
    ## Cross-validation
    train_val_subsets = create_crossval_subsets(front,L90,R90,cd,labels,k)
    fold_loss = []
    
    for j,data in enumerate(train_val_subsets):
        print('Fold {}/{}'.format(j+1, k))
        
        train_data,val_data = data
        train_front,train_L90,train_R90,_,train_labels = train_data
        val_front,val_L90,val_R90,_,val_labels = val_data
        
        n_sick = train_labels.sum()
        n_healthy = len(train_labels) - n_sick
        rate_train = n_healthy / n_sick
        
        # Set randomly initialized weights
        model.set_weights(init_weights)
        
        # Optimizer
        sgd = SGD(learning_rate=lr)

        # Compile the model
        model.compile(loss='binary_crossentropy', metrics=['accuracy', 'AUC', 'TruePositives', 'FalseNegatives', 'TrueNegatives', 'FalsePositives', 'Precision',
                                                           'Recall', Specificity(), WeightedError(rate=20)], optimizer=sgd)

        # Train the model
        history = model.fit([np.expand_dims(train_front,-1), np.expand_dims(train_L90,-1), np.expand_dims(train_R90,-1)], train_labels,
                            class_weight = {0: 1, 1: rate_train}, batch_size=8, epochs=EPOCHS, verbose=0,
                            validation_data=([np.expand_dims(val_front,-1), np.expand_dims(val_L90,-1), np.expand_dims(val_R90,-1)], val_labels)) 
        
        fold_loss.append(history.history['val_loss'][-1])
        
        print()
            
    return np.asarray(fold_loss).mean()  # Objective value linked with the Trial object. 

In [ ]:
# Wrap the objective inside a lambda and call objective inside it
func = lambda trial: objective(trial, model, train_front, train_L90, train_R90, train_cd, train_labels, k, EPOCHS)

In [ ]:
#### 3-block model
print('3-block multi-input CNN for thermal image classification'), print()
classifier_type = 'multiInputCNN_3blocks'

## Initialize model
model = image_classifier_model_3blocks(w, h, 1)

## Tune hyperparameters (lr)
tic = time.time()
study = optuna.create_study()  # Create a new study.
study.optimize(func, n_trials=n_trials)  # Invoke optimization of the objective function.
tuningTime = time.time() - tic
print('Hyperparameter tuning took',str(tuningTime//60),'mins and',str(tuningTime%60),'seconds')
print('Best mean loss achieved:',str(study.best_value))
print()

## Get best hyperparameter combination
lr = study.best_params['lr']
tuned_lr = {classifier_type:lr}

# Visualize the relationship between hyperparameters and the objective value.
fig = optuna.visualization.plot_parallel_coordinate(study, params=['lr'])
fig.write_image(path+'Param_tuning/Param_selection_'+classifier_type+'.png')
fig.show()

# Visualize the objective values over the trials.
optuna.visualization.plot_optimization_history(study)

In [ ]:
#### 1-block model
print('1-block multi-input CNN for thermal image classification'), print()
classifier_type = 'multiInputCNN_1block'

## Initialize model
model = image_classifier_model_1block(w, h)

## Tune hyperparameters (lr)
tic = time.time()
study = optuna.create_study()  # Create a new study.
study.optimize(func, n_trials=n_trials)  # Invoke optimization of the objective function.
tuningTime = time.time() - tic
print('Hyperparameter tuning took',str(tuningTime//60),'mins and',str(tuningTime%60),'seconds')
print('Best mean loss achieved:',str(study.best_value))
print()

## Get best hyperparameter combination
lr = study.best_params['lr']
tuned_lr[classifier_type] = lr

# Visualize the relationship between hyperparameters and the objective value.
fig = optuna.visualization.plot_parallel_coordinate(study, params=['lr'])
fig.write_image(path+'Param_tuning/Param_selection_curve_'+classifier_type+'.png')
fig.show()

# Visualize the objective values over the trials.
optuna.visualization.plot_optimization_history(study)

In [ ]:
print(tuned_lr)

### Train classifiers

In [ ]:
def train_multiInputCNN_classifier(train_front,train_L90,train_R90,train_labels,test_front,test_L90,test_R90,test_labels,blocks=3,lr=0.0001,EPOCHS=30):  
    
    n_sick = train_labels.sum()
    n_healthy = len(train_labels) - n_sick
    rate_train = n_healthy / n_sick

    ## Initialize the model
    if blocks == 3:
        classifier = image_classifier_model_3blocks(w, h, 1)
    else:
        classifier = image_classifier_model_1block(w, h)
        
    ## Train the model
    # Set the optimizer
    sgd = SGD(learning_rate=lr)
    
    # Compile the model
    classifier.compile(loss='binary_crossentropy', metrics=['accuracy', 'AUC', 'TruePositives', 'FalseNegatives', 'TrueNegatives', 'FalsePositives', 'Precision',
                                                            'Recall', Specificity(), WeightedError(rate=20)], optimizer=sgd)

    # Train the model
    tic = time.time()
    history = classifier.fit([np.expand_dims(train_front,-1), np.expand_dims(train_L90,-1), np.expand_dims(train_R90,-1)], train_labels,
                              class_weight = {0: 1, 1: rate_train}, batch_size=8, epochs=EPOCHS, verbose=0,
                              validation_data=([np.expand_dims(test_front,-1), np.expand_dims(test_L90,-1), np.expand_dims(test_R90,-1)], test_labels)) 
    train_time = time.time() - tic

    ## Predictions
    tic = time.time()
    predictions_train = classifier.predict([np.expand_dims(train_front,-1), np.expand_dims(train_L90,-1), np.expand_dims(train_R90,-1)])
    inference_time = time.time() - tic

    tic = time.time()
    predictions_test = classifier.predict([np.expand_dims(test_front,-1), np.expand_dims(test_L90,-1), np.expand_dims(test_R90,-1)])
    inference_time = inference_time + (time.time()-tic)

    inference_time = inference_time/(len(predictions_train)+len(predictions_test))
        
    return classifier,history,train_time,inference_time,predictions_train,predictions_test

In [ ]:
## 3-block multi-input CNN
print('3-block multi-input CNN'), print()
classifier_type = 'multiInputCNN_3blocks'

# Train the model
lr = tuned_lr[classifier_type] # Tuned value
multiInput_3blocks,history,train_time,inference_time,predictions_train,predictions_test = train_multiInputCNN_classifier(train_front,train_L90,train_R90,train_labels,
                                                                                                                         test_front,test_L90,test_R90,test_labels,
                                                                                                                         blocks=3,lr=lr,EPOCHS=EPOCHS)

# Save the model
multiInput_3blocks.save(path+'Models/'+classifier_type+'.h5')

# Save learning curves
plot_training([history.history['loss'],history.history['accuracy'],history.history['auc']],
              [history.history['val_loss'],history.history['val_accuracy'],history.history['val_auc']],EPOCHS)
plt.savefig(path+'Learning_curves/'+classifier_type+'.png'), plt.show()
    
# Save predictions
np.save(path+'Predictions/'+classifier_type+'_train.npy',predictions_train)
np.save(path+'Predictions/'+classifier_type+'_test.npy',predictions_test)

print('TRAIN - ',str(train_time),'s'), print(numeric_results(train_labels,predictions_train))
print('TEST - ', str(inference_time),'s'), print(numeric_results(test_labels,predictions_test))
print()
print('Bootstrapping')
print(bootstrapping(test_labels,predictions_test))
print()
print('Number of total parameters: ', num_parameters_img(multiInput_3blocks)[0])
print('Number of trainable parameters: ', num_parameters_img(multiInput_3blocks)[1])
print()

In [ ]:
## 1-block multi-input CNN
print('1-block multi-input CNN'), print()
classifier_type = 'multiInputCNN_1block'

# Train the model
lr = tuned_lr[classifier_type] # Tuned value
multiInput_1block,history,train_time,inference_time,predictions_train,predictions_test = train_multiInputCNN_classifier(train_front,train_L90,train_R90,train_labels,
                                                                                                                         test_front,test_L90,test_R90,test_labels,
                                                                                                                         blocks=1,lr=lr,EPOCHS=EPOCHS)

# Save the model
multiInput_1block.save(path+'Models/'+classifier_type+'.h5'), plt.show()

# Save learning curves
plot_training([history.history['loss'],history.history['accuracy'],history.history['auc']],
              [history.history['val_loss'],history.history['val_accuracy'],history.history['val_auc']],EPOCHS)
plt.savefig(path+'Learning_curves/'+classifier_type+'.png')
    
# Save predictions
np.save(path+'Predictions/'+classifier_type+'_train.npy',predictions_train)
np.save(path+'Predictions/'+classifier_type+'_test.npy',predictions_test)

print('TRAIN - ',str(train_time),'s'), print(numeric_results(train_labels,predictions_train))
print('TEST - ', str(inference_time),'s'), print(numeric_results(test_labels,predictions_test))
print()
print('Bootstrapping')
print(bootstrapping(test_labels,predictions_test))
print()
print('Number of total parameters: ', num_parameters_img(multiInput_1block)[0])
print('Number of trainable parameters: ', num_parameters_img(multiInput_1block)[1])
print()

## Pre-trained CNNs

In [ ]:
from tensorflow.keras.applications import densenet, inception_v3, mobilenet, mobilenet_v2, vgg16, vgg19, resnet

In [ ]:
CNN_dict = {'densenet121':'densenet.DenseNet121','densenet169':'densenet.DenseNet169','densenet201':'densenet.DenseNet201',
            'mobilenet':'mobilenet.MobileNet','mobilenet_v2':'mobilenet_v2.MobileNetV2',
            'vgg16':'vgg16.VGG16','vgg19':'vgg19.VGG19',
            'resnet50':'resnet.ResNet50','resnet101':'resnet.ResNet101','resnet152':'resnet.ResNet152',
            'inception_v3':'inception_v3.InceptionV3'}

In [ ]:
def pretraindCNN(model_handle, a, b):    
    # Load a pretrained model (e.g., ResNet50)
    baseModel = eval(model_handle + '(include_top=False, input_shape=(a,b,3))')
    
    # Replace top and most task-specifict layer
    model = Sequential()
    model.add(baseModel)
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
        
    return model

In [ ]:
## 3-block model, one for each view
def pretraindCNN_3blocks(model_handle,a,b):
    
    ## Front
    front_input = Input(shape=(a, b, 1))
    front = tf.repeat(front_input,3,-1)  # Convert to 3-channel image
    front_model = pretraindCNN(model_handle, a, b)
    front_model._name = 'front'
    front = front_model(front)
    
    ## L90
    L90_input = Input(shape=(a, b, 1))
    L90 = tf.repeat(L90_input,3,-1)  # Convert to 3-channel image
    L90_model = pretraindCNN(model_handle, a, b)
    L90_model._name = 'L90'
    L90 = L90_model(L90)
    
    ## R90
    R90_input = Input(shape=(a, b, 1))
    R90 = tf.repeat(R90_input,3,-1)  # Convert to 3-channel image
    R90_model = pretraindCNN(model_handle, a, b)
    R90_model._name = 'R90'
    R90 = R90_model(R90)
    
    ## Ensemble
    concatenated = concatenate([front, L90, R90], axis=-1)

    X = Dense(1, activation='sigmoid')(concatenated)   

    ## create the model
    model = Model([front_input, L90_input, R90_input], X) 
    
    return model

In [ ]:
## 1-block model, with each view as an input channel
def pretraindCNN_1block(model_handle,a,b):
    # Inputs
    front_input = Input(shape=(a, b, 1))
    L90_input = Input(shape=(a, b, 1))
    R90_input = Input(shape=(a, b, 1))
    stacked_input = tf.concat([front_input, L90_input, R90_input], axis=-1)
    
    # Model
    model = pretraindCNN(model_handle, a, b)
    X = model(stacked_input)
    
    ## create the model
    model = Model([front_input, L90_input, R90_input], X)  

    return model

In [ ]:
def freezeLayers(model,perc_unfreeze,baseModel_name):
    for layer in model.layers:
        if layer.name in ['front','L90','R90']:  # 3-block model
            for layer2 in layer.layers:
                if layer2.name == baseModel_name:
                    L = len(layer2.layers)
                    L_unfreeze = round(L*perc_unfreeze)
                    for i,layer3 in enumerate(reversed(layer2.layers)):
                        if (i+1) <= L_unfreeze:
                            layer3.trainable = True
                        else:
                            layer3.trainable = False
                else:
                    layer2.trainable = True
                    
        elif layer.name == baseModel_name:  # 1-block model
            L = len(layer.layers)
            L_unfreeze = round(L*perc_unfreeze)
            for i,layer2 in enumerate(reversed(layer.layers)):
                if (i+1) <= L_unfreeze:
                    layer2.trainable = True
                else:
                    layer2.trainable = False
            else:
                layer.trainable = True
        else:
            layer.trainable = True
            
    return model

In [ ]:
def finetune(model,baseModel_name,train_front,train_L90,train_R90,train_labels,test_front,test_L90,test_R90,test_labels,
             lr_freeze,lr_unfreeze,perc_unfreeze,EPOCHS):
    ## Step 1: train randomly initialized weights
    #num_epochs_freeze = num_epochs//2
    print('Freezing base model...'), print()
    
    n_sick = train_labels.sum()
    n_healthy = len(train_labels) - n_sick
    rate_train = n_healthy / n_sick
    
    # Optimizer
    sgd = SGD(learning_rate=lr_freeze)
    
    # Freeze the base pretrained CNN
    model = freezeLayers(model,0,baseModel_name)

    # Compile the model
    model.compile(loss='binary_crossentropy', metrics=['accuracy', 'AUC', 'TruePositives', 'FalseNegatives', 'TrueNegatives', 'FalsePositives', 'Precision',
                                                       'Recall', Specificity(), WeightedError(rate=20)], optimizer=sgd)
    
    # Train the model
    history = model.fit([np.expand_dims(train_front,-1), np.expand_dims(train_L90,-1), np.expand_dims(train_R90,-1)], train_labels,
                        class_weight={0: 1, 1: rate_train}, batch_size=8, epochs=EPOCHS//2, verbose=0,
                        validation_data=([np.expand_dims(test_front,-1), np.expand_dims(test_L90,-1), np.expand_dims(test_R90,-1)], test_labels)) 
    
    train_loss = history.history['loss']
    train_acc = history.history['accuracy']
    train_auc = history.history['auc']
    val_loss = history.history['val_loss']
    val_acc = history.history['val_accuracy']
    val_auc = history.history['val_auc']
    
    print('Step 1 completed.'), print()
    
    ## Step 2: fine tune all the parameters in the model
    print('Unfreezing base model...'), print()
    
    # Optimizer
    sgd = SGD(learning_rate=lr_unfreeze)
    
    # Unfreeze the top perc_unfreeze% layers in the encoder
    model = freezeLayers(model,perc_unfreeze,baseModel_name)
    
    # Compile the model
    model.compile(loss='binary_crossentropy', metrics=['accuracy', 'AUC', 'TruePositives', 'FalseNegatives', 'TrueNegatives', 'FalsePositives', 'Precision',
                                                       'Recall', Specificity(), WeightedError(rate=20)], optimizer=sgd)
    
    # Train the model
    history = model.fit([np.expand_dims(train_front,-1), np.expand_dims(train_L90,-1), np.expand_dims(train_R90,-1)], train_labels,
                        class_weight={0: 1, 1: rate_train}, batch_size=8, epochs=EPOCHS//2, verbose=0,
                        validation_data=([np.expand_dims(test_front,-1), np.expand_dims(test_L90,-1), np.expand_dims(test_R90,-1)], test_labels)) 
    
    train_loss = train_loss + history.history['loss']
    train_acc = train_acc + history.history['accuracy']
    train_auc = train_auc + history.history['auc']
    val_loss = val_loss + history.history['val_loss']
    val_acc = val_acc + history.history['val_accuracy']
    val_auc = val_auc + history.history['val_auc']
    
    print('Step 2 completed.'), print()
    
    train_history = [train_loss,train_acc,train_auc]
    val_history = [val_loss,val_acc,val_auc]
    
    return model, [train_history, val_history]

In [ ]:
# Percentage of top layers to unfreeze during second step of fine-tuning
perc_unfreeze = 0.2

### Tune hyperparameters

In [ ]:
def objective(trial, model, front, L90, R90, cd, labels, k, EPOCHS=30, perc_unfreeze=0.2):  
    
    ## Initialize model
    init_weights = model.get_weights()
    
    ## Hyperparameters
    # Learning rates
    lr_freeze = trial.suggest_float('lr_freeze', 1e-5, 1e-2, log=True)
    lr_unfreeze = trial.suggest_float('lr_unfreeze', 1e-6, 1e-2, log=True)
    
    ## Cross-validation
    train_val_subsets = create_crossval_subsets(front,L90,R90,cd,labels,k)
    fold_loss = []
    
    for j,data in enumerate(train_val_subsets):
        print('Fold {}/{}'.format(j+1, k))
        
        train_data,val_data = data
        train_front,train_L90,train_R90,_,train_labels = train_data
        val_front,val_L90,val_R90,_,val_labels = val_data
        
        # Set randomly initialized weights
        model.set_weights(init_weights)
        
        # Train the model
        model, history = finetune(model,baseModel_name,train_front,train_L90,train_R90,train_labels,test_front,test_L90,test_R90,test_labels,
                                  lr_freeze,lr_unfreeze,perc_unfreeze,EPOCHS)
        _, val_history = history
        val_loss,_,_ = val_history
        
        fold_loss.append(val_loss[-1])
                    
    return np.asarray(fold_loss).mean()  # Objective value linked with the Trial object. 

In [ ]:
# Wrap the objective inside a lambda and call objective inside it
func_PT = lambda trial: objective(trial, model, train_front, train_L90, train_R90, train_cd, train_labels, k, EPOCHS, perc_unfreeze)

In [ ]:
tuned_lr = {}

## 3-CNN model
for baseModel_name in CNN_dict.keys():
    print(baseModel_name), print()
    model_handle = CNN_dict[baseModel_name]
    
    #### 3-block model
    print('Model with 3 pre-trained CNNs for thermal image classification'), print()
    classifier_type = baseModel_name+'_3blocks'

    ## Initialize model
    model = pretraindCNN_3blocks(model_handle,w,h)

    ## Tune hyperparameters (lr_freeze and lr_unfreeze)
    tic = time.time()
    study = optuna.create_study()  # Create a new study.
    study.optimize(func_PT, n_trials=n_trials)  # Invoke optimization of the objective function.
    tuningTime = time.time() - tic
    
    print('Hyperparameter tuning took',str(tuningTime//60),'mins and',str(tuningTime%60),'seconds')
    print('Best mean loss achieved:',str(study.best_value))
    print()

    ## Get best hyperparameter combination
    lr_freeze = study.best_params['lr_freeze']
    lr_unfreeze = study.best_params['lr_unfreeze']
    tuned_lr[classifier_type] = [lr_freeze,lr_unfreeze]

    # Visualize the relationship between hyperparameters and the objective value.
    fig = optuna.visualization.plot_parallel_coordinate(study, params=['lr_freeze','lr_unfreeze'])
    fig.write_image(path+'Param_tuning/Param_selection_pretrained_'+classifier_type+'.png')
    fig.show()

    # Visualize the objective values over the trials.
    optuna.visualization.plot_optimization_history(study)
    
    print(),print('#'*150),print()


In [ ]:
## 1-CNN model
for baseModel_name in CNN_dict.keys():
    print(baseModel_name), print()
    model_handle = CNN_dict[baseModel_name]
    
    #### 1-block model
    print('Model with 1 pre-trained CNN for thermal image classification'), print()
    classifier_type = baseModel_name+'_1block'

    ## Initialize model
    model = pretraindCNN_1block(model_handle,w,h)

    ## Tune hyperparameters (lr_freeze and lr_unfreeze)
    tic = time.time()
    study = optuna.create_study()  # Create a new study.
    study.optimize(func_PT, n_trials=n_trials)  # Invoke optimization of the objective function.
    tuningTime = time.time() - tic
    
    print('Hyperparameter tuning took',str(tuningTime//60),'mins and',str(tuningTime%60),'seconds')
    print('Best mean loss achieved:',str(study.best_value))
    print()

    ## Get best hyperparameter combination
    lr_freeze = study.best_params['lr_freeze']
    lr_unfreeze = study.best_params['lr_unfreeze']
    tuned_lr[classifier_type] = [lr_freeze,lr_unfreeze]

    # Visualize the relationship between hyperparameters and the objective value.
    fig = optuna.visualization.plot_parallel_coordinate(study, params=['lr_freeze','lr_unfreeze'])
    fig.write_image(path+'Param_tuning/Param_selection_pretrained_'+classifier_type+'.png')
    fig.show()

    # Visualize the objective values over the trials.
    optuna.visualization.plot_optimization_history(study)
    
    print(),print('#'*150),print()

In [ ]:
print(tuned_lr)

### Train classifiers

In [ ]:
def train_pretrainedCNN_classifier(baseModel_name,model_handle,train_front,train_L90,train_R90,train_labels,test_front,test_L90,test_R90,test_labels,
                                   blocks=3,lr_freeze=0.0001,lr_unfreeze=0.00001,perc_unfreeze=0.2,EPOCHS=30):  
    
    ## Initialize the model
    if blocks == 3:
        classifier = pretraindCNN_3blocks(model_handle,w,h)
    else:
        classifier = pretraindCNN_1block(model_handle,w,h)
        
    # Train the model
    tic = time.time()
    classifier, history = finetune(classifier,baseModel_name,train_front,train_L90,train_R90,train_labels,test_front,test_L90,test_R90,test_labels,
                                   lr_freeze,lr_unfreeze,perc_unfreeze,EPOCHS) 
    train_time = time.time() - tic

    ## Predictions
    tic = time.time()
    predictions_train = classifier.predict([np.expand_dims(train_front,-1), np.expand_dims(train_L90,-1), np.expand_dims(train_R90,-1)])
    inference_time = time.time() - tic

    tic = time.time()
    predictions_test = classifier.predict([np.expand_dims(test_front,-1), np.expand_dims(test_L90,-1), np.expand_dims(test_R90,-1)])
    inference_time = inference_time + (time.time()-tic)

    inference_time = inference_time/(len(predictions_train)+len(predictions_test))
        
    return classifier,history,train_time,inference_time,predictions_train,predictions_test

In [ ]:
## 3-CNN model
for baseModel_name in CNN_dict.keys():
    print(baseModel_name), print()
    model_handle = CNN_dict[baseModel_name]
    classifier_type = baseModel_name+'_3blocks'
    
    ## Tuned parameters
    lr_freeze = tuned_lr[classifier_type][0]
    lr_unfreeze = tuned_lr[classifier_type][1]
        
    ## Train the model
    pretrained_3blocks,history,train_time,inference_time,predictions_train,predictions_test = train_pretrainedCNN_classifier(baseModel_name,model_handle,
                                                                                                                             train_front,train_L90,train_R90,train_labels,
                                                                                                                             test_front,test_L90,test_R90,test_labels,
                                                                                                                             3,lr_freeze,lr_unfreeze,perc_unfreeze,EPOCHS)
        
    # Save history
    train_history, val_history = history
    plot_training(train_history,val_history,EPOCHS)
    plt.savefig(path+'Learning_curves/'+classifier_type+'.png'), plt.show()

    # Save model
    pretrained_3blocks.save(path+'Models/'+classifier_type+'.h5')
    
    # Save predictions
    np.save(path+'Predictions/'+classifier_type+'_train.npy',predictions_train)
    np.save(path+'Predictions/'+classifier_type+'_test.npy',predictions_test)
    
    print('TRAIN - ',str(train_time),'s'), print(numeric_results(train_labels,predictions_train))
    print('TEST - ', str(inference_time),'s'), print(numeric_results(test_labels,predictions_test))
    print()
    print('Bootstrapping')
    print(bootstrapping(test_labels,predictions_test))
    print()
    print('Number of total parameters: ', num_parameters_img(pretrained_3blocks)[0])
    print('Number of trainable parameters: ', num_parameters_img(pretrained_3blocks)[1])
    print()

In [ ]:
## 1-CNN model
for baseModel_name in CNN_dict.keys():
    print(baseModel_name), print()
    model_handle = CNN_dict[baseModel_name]
    classifier_type = baseModel_name+'_1blocks'
    
    ## Tuned parameters
    lr_freeze = tuned_lr[classifier_type][0]
    lr_unfreeze = tuned_lr[classifier_type][1]
        
    ## Train the model
    pretrained_1block,history,train_time,inference_time,predictions_train,predictions_test = train_pretrainedCNN_classifier(baseModel_name,model_handle,
                                                                                                                            train_front,train_L90,train_R90,train_labels,
                                                                                                                            test_front,test_L90,test_R90,test_labels,
                                                                                                                            1,lr_freeze,lr_unfreeze,perc_unfreeze,EPOCHS)
        
    # Save history
    train_history, val_history = history
    plot_training(train_history,val_history,EPOCHS)
    plt.savefig(path+'Learning_curves/'+classifier_type+'.png'), plt.show()

    # Save model
    pretrained_1block.save(path+'Models/'+classifier_type+'.h5')
    
    # Save predictions
    np.save(path+'Predictions/'+classifier_type+'_train.npy',predictions_train)
    np.save(path+'Predictions/'+classifier_type+'_test.npy',predictions_test)
    
    print('TRAIN - ',str(train_time),'s'), print(numeric_results(train_labels,predictions_train))
    print('TEST - ', str(inference_time),'s'), print(numeric_results(test_labels,predictions_test))
    print()
    print('Bootstrapping')
    print(bootstrapping(test_labels,predictions_test))
    print()
    print('Number of total parameters: ', num_parameters_img(pretrained_1block)[0])
    print('Number of trainable parameters: ', num_parameters_img(pretrained_1block)[1])
    print()

## Compare with segmented frontal images

In [ ]:
path_segmentation = ... # Set the path where the segmentation models and results are stored

In [ ]:
segmentation_results = pd.read_csv(path_segmentation+'Results.csv',header=0)
best_model = segmentation_results.iloc[np.argmax(segmentation_results['testDice'].values)]['pretrainedCNN']
segmentation_model = tf.keras.models.load_model(path_segmentation+'Models/'+best_model+'.h5', compile = False)

In [ ]:
dsize = segmentation_model.input.shape

In [ ]:
## Apply segmentation model to frontal images

# Train images
train_front_reduced = np.asarray([cv2.resize(img,(dsize[2],dsize[1])) for img in train_front])  # Reduce image size to 240x320
train_front_masks = segmentation_model(np.repeat(np.expand_dims(train_front_reduced,axis=-1),3,axis=-1))
train_front_masks = np.asarray([cv2.resize(tf.squeeze(mask).numpy(),(train_front.shape[2],train_front.shape[1])) for mask in train_front_masks])  # Resize mask to original image shape
train_front_segmented = np.multiply(train_front,train_front_masks)

# Test images
test_front_reduced = np.asarray([cv2.resize(img,(dsize[2],dsize[1])) for img in test_front])  # Reduce image size to 240x320
test_front_masks = segmentation_model(np.repeat(np.expand_dims(test_front_reduced,axis=-1),3,axis=-1))
test_front_masks = np.asarray([cv2.resize(tf.squeeze(mask).numpy(),(test_front.shape[2],test_front.shape[1])) for mask in test_front_masks])  # Resize mask to original image shape
test_front_segmented = np.multiply(test_front,test_front_masks)

# Display some examples
N = 10
idx = [np.random.randint(0,len(train_front)) for i in range(N)]
display_list = [[train_front[i],train_front_masks[i]] for i in idx]
plt.figure(figsize=(5*N//2,8))
for i in range(N):
    img = display_list[i][0]
    mask = display_list[i][1]
    plt.subplot(2, N//2, i+1)
    plt.imshow(img,'gray'), plt.imshow(mask,alpha=0.5)            
    plt.axis('off')
plt.savefig(path+'Example_segmented_images.png')
plt.show()


## Remove background in frontal and lateral images
# Plot the histogram to select threshold
plt.figure(figsize=(10, 6))
plt.hist(np.concatenate((train_front,train_L90,train_R90,test_front,test_L90,test_R90)).flatten(), 
         bins=256, range=(0, 1), density=True, color='blue', alpha=0.7)
plt.title('Histogram of Pixel Values')
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')
plt.show()

#threshold = input('Enter threshold value: ')
#threshold = float(threshold)
threshold = 0.41

# Apply threshold to remove background
train_front_segmented = np.multiply(train_front_segmented,train_front_segmented>threshold)
test_front_segmented = np.multiply(test_front_segmented,test_front_segmented>threshold)
train_L90_foreground = np.multiply(train_L90,train_L90>threshold)
test_L90_foreground = np.multiply(test_L90,test_L90>threshold)
train_R90_foreground = np.multiply(train_R90,train_R90>threshold)
test_R90_foreground = np.multiply(test_R90,test_R90>threshold)

### Tune hyperparameters

In [ ]:
def objective(trial, model, front, L90, R90, cd, labels, k, EPOCHS=30):  
    
    ## Initialize model
    init_weights = model.get_weights()
    
    ## Hyperparameters
    # Learning rate
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    
    ## Cross-validation
    train_val_subsets = create_crossval_subsets(front,L90,R90,cd,labels,k)
    fold_loss = []
    
    for j,data in enumerate(train_val_subsets):
        print('Fold {}/{}'.format(j+1, k))
        
        train_data,val_data = data
        train_front,train_L90,train_R90,_,train_labels = train_data
        val_front,val_L90,val_R90,_,val_labels = val_data
        
        n_sick = train_labels.sum()
        n_healthy = len(train_labels) - n_sick
        rate_train = n_healthy / n_sick
        
        # Set randomly initialized weights
        model.set_weights(init_weights)
        
        # Optimizer
        sgd = SGD(learning_rate=lr)

        # Compile the model
        model.compile(loss='binary_crossentropy', metrics=['accuracy', 'AUC', 'TruePositives', 'FalseNegatives', 'TrueNegatives', 'FalsePositives', 'Precision',
                                                           'Recall', Specificity(), WeightedError(rate=20)], optimizer=sgd)

        # Train the model
        history = model.fit([np.expand_dims(train_front,-1), np.expand_dims(train_L90,-1), np.expand_dims(train_R90,-1)], train_labels,
                            class_weight={0: 1, 1: rate_train}, batch_size=8, epochs=EPOCHS, verbose=0,
                            validation_data=([np.expand_dims(val_front,-1), np.expand_dims(val_L90,-1), np.expand_dims(val_R90,-1)], val_labels)) 
        
        fold_loss.append(history.history['val_loss'][-1])
        
        print()
            
    return np.asarray(fold_loss).mean()  # Objective value linked with the Trial object. 

In [ ]:
# Wrap the objective inside a lambda and call objective inside it
func = lambda trial: objective(trial, model, train_front_segmented, train_L90_foreground ,train_R90_foreground, train_cd, train_labels, k, EPOCHS)

In [ ]:
classifier_type = 'multiInputCNN_3blocks_segmented'

## Initialize model
model = image_classifier_model_3blocks(w, h, 1)

## Tune hyperparameters (lr)
tic = time.time()
study = optuna.create_study()  # Create a new study.
study.optimize(func, n_trials=n_trials)  # Invoke optimization of the objective function.
tuningTime = time.time() - tic

## Get best hyperparameter combination
lr = study.best_params['lr']
tuned_lr = {classifier_type:lr}

# Visualize the relationship between hyperparameters and the objective value.
fig = optuna.visualization.plot_parallel_coordinate(study, params=['lr'])
fig.write_image(path+'Param_tuning/Param_selection_curve_'+classifier_type+'.png')
fig.show()

# Visualize the objective values over the trials.
optuna.visualization.plot_optimization_history(study)


In [ ]:
print(tuned_lr)

### Train classifiers

In [ ]:
classifier_type = 'multiInputCNN_3blocks_segmented'

# Train the model
lr = tuned_lr[classifier_type] # Tuned value
multiInput_3blocks_segmented,history,train_time,inference_time,predictions_train,predictions_test = train_multiInputCNN_classifier(train_front_segmented,train_L90_foreground,train_R90_foreground,train_labels,
                                                                                                                                   test_front_segmented,test_L90_foreground,test_R90_foreground,test_labels,
                                                                                                                                   blocks=3,lr=lr,EPOCHS=EPOCHS)

# Save the model
multiInput_3blocks_segmented.save(path+'Models/'+classifier_type+'.h5')

# Save learning curves
plot_training([history.history['loss'],history.history['accuracy'],history.history['auc']],
              [history.history['val_loss'],history.history['val_accuracy'],history.history['val_auc']],EPOCHS)
plt.savefig(path+'Learning_curves/'+classifier_type+'.png'), plt.show()
    
# Save predictions
np.save(path+'Predictions/'+classifier_type+'_train.npy',predictions_train)
np.save(path+'Predictions/'+classifier_type+'_test.npy',predictions_test)

print('TRAIN - ',str(train_time),'s'), print(numeric_results(train_labels,predictions_train))
print('TEST - ', str(inference_time),'s'), print(numeric_results(test_labels,predictions_test))
print()
print('Bootstrapping')
print(bootstrapping(test_labels,predictions_test))
print()
print('Number of total parameters: ', num_parameters_img(multiInput_3blocks_segmented)[0])
print('Number of trainable parameters: ', num_parameters_img(multiInput_3blocks_segmented)[1])
print()

## Select best classifier

In [ ]:
print('Best model to classify thermal images is multi-input CNN with 3 blocks, with one block for each view.')
img_model = 'multiInputCNN_3blocks'

# Ensemble model

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier, Perceptron
import pickle

In [ ]:
def tune_weight(pred_CD, pred_img, labels, metric=None):
    best_weight = 0
    best_metric = 0
    if metric == "we":
        best_metric = np.inf
            
    for weight_cd in range(0, 10000, 1):
        weight = weight_cd/10000
                
        new_pred = pred_CD * weight + pred_img * (1 - weight)
        
        if metric == "accuracy":
            accuracy = accuracy_score(labels, np.round(new_pred))
            if accuracy > best_metric:
                best_metric = accuracy
                best_weight = weight
        elif metric == "roc_auc":
            roc_auc = roc_auc_score(labels, new_pred)
            if roc_auc > best_metric:
                best_metric = roc_auc
                best_weight = weight
        elif metric == "we":
            we = funcion_perdida_we(labels, np.round(new_pred))
            if we < best_metric:
                best_metric = we
                best_weight = weight
        else:
            tune_weight(pred_CD, pred_img, labels, "accuracy")
            tune_weight(pred_CD, pred_img, labels, "roc_auc")
            tune_weight(pred_CD, pred_img, labels, "we")
            break
            
    print("\Best ", metric , ": ", best_metric, " | Best weight: ", best_weight)
    return best_weight


def weighted_voting(pred_img,pred_cd,weight=0.698):
    return pred_cd * weight + pred_img * (1 - weight)

In [ ]:
models = ['WeightedVoting','DT','SGD','NN','SVM']

In [ ]:
## Read predictions
# Clinical data classifier
pred_CD_train = np.load(path+'Predictions/cd_'+cd_model+'_train.npy')
pred_CD_test = np.load(path+'Predictions/cd_'+cd_model+'_test.npy')
    
# Clinical data classifier
pred_img_train = np.load(path+'Predictions/'+img_model+'_train.npy')
pred_img_test = np.load(path+'Predictions/'+img_model+'_test.npy')

# Concatenate predictions to generate the input vector
inputs_train = np.concatenate((pred_img_train.reshape(-1, 1),pred_CD_train.reshape(-1, 1)), axis=1)
inputs_test = np.concatenate((pred_img_test.reshape(-1, 1),pred_CD_test.reshape(-1, 1)), axis=1)

## Tune hyperparameters

In [ ]:
ensemble_params = {}

In [ ]:
# Tune weight for weighted voting
tic = time.time()
weight = tune_weight(pred_CD_train, pred_img_train[:,0], train_labels, metric='accuracy')
train_time_wv = time.time() - tic
print('Tuned weight:',weight)
ensemble_params['WeightedVoting'] = {'weight' : weight}

# Evaluate
pred_ensemble_train = weighted_voting(pred_img_train[:,0],pred_CD_train,weight)
print('TRAIN'), print(numeric_results(train_labels,pred_ensemble_train)), print()
pred_ensemble_test = weighted_voting(pred_img_test[:,0],pred_CD_test,weight)
print('TEST:'), print(numeric_results(test_labels,pred_ensemble_test)), print()

In [ ]:
## Tune other classifier's parameters with 4-fold cross-validation
# Tune models
for classifier_type in models[1::]:
    print(classifier_type), print()
    
    if classifier_type == 'DT':
        estimator = DecisionTreeClassifier(class_weight={0: 1, 1: rate_train})
        param_grid = {'ccp_alpha' : np.arange(0, 0.1, 0.005),
                      'criterion': ['gini', 'entropy'],  # Function to measure the quality of a split
                      'max_depth' : [None, 10, 20, 30, 40, 50], #[None, 1, 10, 15],  # Maximum depth of the tree
                      'max_features': [None, 'auto', 'sqrt', 'log2'], #['auto'],  # Number of features to consider when looking for the best split
                      'max_leaf_nodes': [None, 10, 20, 30, 40, 50], #[None, 3, 6, 9],  # Grow a tree with max_leaf_nodes in best-first fashion
                      'min_samples_leaf': [1, 5, 10, 20], #[2, 3, 4],  # Minimum number of samples required to be at a leaf node
                      'min_samples_split' : [2, 10, 20, 30], #[3, 10, 15],  # Minimum number of samples required to split an internal node
                      'min_weight_fraction_leaf' : np.arange(0.1, 0.5, 0.01),
                      'splitter': ['best', 'random'],    # Strategy used to choose the split at each node
                     }        
    elif classifier_type == 'SVM':
        estimator = SVC(class_weight={0: 1, 1: rate_train}, probability=True)
        param_grid = {'C': [0.1, 1, 10, 100], # range(1, 1000, 10), 
                      'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 
                      #'degree': [2, 3, 4, 5],
                      #'coef0': [0.0, 0.1, 0.5, 1.0],
                      #'shrinking': [True, False],
                      #'decision_function_shape': ['ovo', 'ovr'],
                      'gamma': ['scale', 'auto', 0.001,  0.01, 0.1, 1], # 0.0001, 0.00001]
                     }
    elif classifier_type == 'SGD':
        estimator = SGDClassifier(class_weight={0: 1, 1: rate_train})
        param_grid = {'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
                      'penalty': [None, 'l2', 'l1', 'elasticnet'],
                      'alpha': [0.0001, 0.001, 0.01, 0.1], #, 1, 10],
                      'l1_ratio': [0.15, 0.3, 0.5, 0.7, 0.9],  # Only used if penalty is 'elasticnet'
                      'max_iter': [1000, 2000, 3000],
                      'tol': [1e-3, 1e-4, 1e-5],
                      'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
                      'eta0':  [0.001, 0.01, 0.1] #, 1, 10]  # Only used if learning_rate is 'constant', 'invscaling', or 'adaptive'
                     }
    else:
        estimator = Perceptron(class_weight={0: 1, 1: rate_train})
        param_grid = {'penalty': [None, 'l2', 'l1', 'elasticnet'],
                      'alpha': [0.0001, 0.001, 0.01, 0.1, 1],
                      'fit_intercept': [True, False],
                      'max_iter': [1000, 2000, 3000, 4000, 5000],
                      'tol': [1e-3, 1e-4, 1e-5],
                      'shuffle': [True, False],
                      'eta0': [0.1, 0.01, 0.001],
                      'early_stopping': [True, False],
                      'validation_fraction': [0.1, 0.2, 0.3],
                      'n_iter_no_change': [5, 10, 20]
                     }
        #estimator = MLPClassifier(random_state = 1)
        #param_grid = {'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 100)],
        #              'activation': ['tanh', 'relu', 'logistic'],
        #              'solver': ['sgd', 'adam'],
        #              'alpha': [0.0001, 0.001, 0.01, 0.1],
        #              'learning_rate': ['constant', 'adaptive'],
        #              'learning_rate_init': [0.001, 0.01, 0.1],
        #              'max_iter': [200, 500],
        #              'batch_size': ['auto', 32, 64, 128],
        #              'tol': [1e-4, 1e-3, 1e-2]
        #             }
    
    classifier, parameters, results_cv = classifier_tuneHyperparameters(estimator, param_grid, inputs_train, train_labels)
    ensemble_params[classifier_type] = parameters
    
    print('Tuned parameters:'), print(parameters), print()
    
    # Evaluate
    print('TRAIN:'), print(numeric_results(train_labels,classifier.predict_proba(inputs_train)))
    print('TEST:'), print(numeric_results(test_labels,classifier.predict_proba(inputs_test)))
    
    print('#'*150), print()

## Train classifiers

In [ ]:
def train_ensemble_classifier(inputs_train,train_labels,inputs_test,test_labels,classifier_type):  
    
    n_sick = train_labels.sum()
    n_healthy = len(train_labels) - n_sick
    rate_train = n_healthy / n_sick
    
    ## Initialize the estimator
    if classifier_type == 'SVM':
        # Support Vector Machine (SVM) classifier
        parameters = ensemble_params[classifier_type]
        #parameters = {'C':1.0, 'kernel':'rbf', 'gamma':'scale'}
        estimator = SVC(class_weight={0: 1, 1: rate_train}, probability=True)
        
    elif classifier_type == 'DT':
        # Decision Tree (DT) classifier
        parameters = ensemble_params[classifier_type]
        #parameters = {'ccp_alpha': 0.0, 'criterion': 'gini', 'max_depth': 10, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_samples_leaf': 3, 
        #              'min_samples_split': 10, 'min_weight_fraction_leaf': 0.13999999999999999, 'splitter': 'best'}
        estimator = DecisionTreeClassifier(class_weight={0: 1, 1: rate_train})
            
    elif classifier_type == 'SGD':
        # Linear classifier
        parameters = ensemble_params[classifier_type]
        #parameters = {'loss': 'modified_huber'}
        estimator = SGDClassifier(class_weight={0: 1, 1: rate_train})
            
    else:
        # Neural Network
        parameters = ensemble_params[classifier_type]
        estimator = Perceptron(class_weight={0: 1, 1: rate_train})
        #estimator = MLPClassifier(random_state = 1)        
            
    estimator.set_params(**parameters)
    
        
    ## Train the classifier
    tic = time.time()
    classifier = estimator.fit(inputs_train, train_labels)
    train_time = time.time() - tic

    ## Predictions
    tic = time.time()
    if classifier_type == 'NN':
        predictions_train = classifier.predict(inputs_train)
        predictions_test = classifier.predict(inputs_test)
    else:
        predictions_train = classifier.predict_proba(inputs_train)[:,1]
        predictions_test = classifier.predict_proba(inputs_test)[:,1]
    inference_time = (time.time()-tic)/(len(predictions_train)+len(predictions_test))
        
    return classifier,train_time,inference_time,predictions_train,predictions_test

In [ ]:
## Weighted voting classifier
print('WeightedVoting'), print()
# Train the model
tic = time.time()
weight = tune_weight(pred_CD_train, pred_img_train[:,0], train_labels, metric='accuracy')
train_time = time.time() - tic
# Evaluate
tic = time.time()
predictions_train = weighted_voting(pred_img_train[:,0],pred_CD_train,weight)
predictions_test = weighted_voting(pred_img_test[:,0],pred_CD_test,weight)
inference_time = (time.time()-tic)/(len(predictions_train)+len(predictions_test))

np.save(path+'Predictions/ensemble_WeightedVoting_train.npy',predictions_train)
np.save(path+'Predictions/ensemble_WeightedVoting_test.npy',predictions_test)

print('TRAIN - ',str(train_time),'s'), print(numeric_results(train_labels,predictions_train))
print('TEST - ', str(inference_time),'s'), print(numeric_results(test_labels,predictions_test))
print()
print('Bootstrapping')
print(bootstrapping(test_labels,predictions_test))
print()
print('Number of parameters: ', num_parameters_classifiers(classifier_type,classifier))
print()




## Train classifiers
for classifier_type in models[1::]:
    print(classifier_type), print()
        
    ## Train the classifier
    classifier,train_time,inference_time,predictions_train,predictions_test = train_ensemble_classifier(inputs_train,train_labels,
                                                                                                        inputs_test,test_labels,
                                                                                                        classifier_type, ensemble_params)
    
    ## Save the model
    with open(path+'Models/ensemble_'+classifier_type+'.pkl','wb') as f:
        pickle.dump(classifier,f)

    ## Store predictions
    np.save(path+'Predictions/ensemble_'+classifier_type+'_trai.npy',predictions_train)
    np.save(path+'Predictions/ensemble_'+classifier_type+'_test.npy',predictions_test)
    
    print('TRAIN - ',str(train_time),'s'), print(numeric_results(train_labels,predictions_train))
    print('TEST - ', str(inference_time),'s'), print(numeric_results(test_labels,predictions_test))
    print()
    print('Bootstrapping')
    print(bootstrapping(test_labels,predictions_test))
    print()
    print('Number of parameters: ', num_parameters_classifiers(classifier_type,classifier))
    print()


## Select best classifier

In [ ]:
ensemble_model = 'SVM'